**Необходимые импорты**

In [164]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import xgboost as xgb

**Загружаем датасет**

In [165]:
data = pd.read_csv('spaceship_titanic.csv').dropna()
print(data.columns)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')


**Обрабатываем датасет**

In [166]:
data = data.drop(['PassengerId', 'Name'], axis=1)

categorical_features = ['HomePlanet', 'CryoSleep', 'Cabin', 
                     'Destination', 'VIP', 'Transported']
numeric_features = ['Age', 'RoomService', 'FoodCourt', 
                    'ShoppingMall', 'Spa', 'VRDeck']

# Категориальные признаки
for feature in categorical_features:
    label_encoder = LabelEncoder()
    data[feature] = label_encoder.fit_transform(data[feature])

# Числовые признаки
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

print(data)

      HomePlanet  CryoSleep  Cabin  Destination       Age  VIP  RoomService  \
0              1          0    137            2  0.695413    0    -0.345756   
1              0          0   1823            2 -0.336769    0    -0.176748   
2              1          0      1            2  2.002842    1    -0.279083   
3              1          0      1            2  0.282540    0    -0.345756   
4              0          0   1825            2 -0.887266    0     0.124056   
...          ...        ...    ...          ...       ...  ...          ...   
8688           1          0    134            0  0.833037    1    -0.345756   
8689           0          1   4293            1 -0.749641    0    -0.345756   
8690           0          0   4298            2 -0.199145    0    -0.345756   
8691           1          0   1778            0  0.213728    0    -0.345756   
8692           1          0   1778            2  1.039473    0    -0.150389   

      FoodCourt  ShoppingMall       Spa    VRDeck  

**Подготавливаем к обучению, делим на train и test**

In [167]:
x = data.drop(['Transported'], axis=1)
y = data['Transported']

x = torch.Tensor(x.values)
y = torch.Tensor(y.values)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

**Torch штуки**

In [168]:
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

**Модели (2, 4 и 8 слоев)**

In [169]:
class Classifier2L(nn.Module):
    def __init__(self, input_size, layer_size, activation_func):
        super().__init__()
        
        if activation_func == 'relu':
            self.activation_func = nn.ReLU()
        elif activation_func == 'sigmoid':
            self.activation_func = nn.Sigmoid()
        
        self.fc1 = nn.Linear(input_size, layer_size)
        self.fc2 = nn.Linear(layer_size, 1)

    def forward(self, x):
        x = self.activation_func(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [170]:
class Classifier4L(nn.Module):
    def __init__(self, input_size, layer_size, activation_func):
        super().__init__()
        
        if activation_func == 'relu':
            self.activation_func = nn.ReLU()
        elif activation_func == 'sigmoid':
            self.activation_func = nn.Sigmoid()
        
        self.fc1 = nn.Linear(input_size, layer_size)
        self.fc2 = nn.Linear(layer_size, layer_size)
        self.fc3 = nn.Linear(layer_size, layer_size)
        self.fc4 = nn.Linear(layer_size, 1)

    def forward(self, x):
        x = self.activation_func(self.fc1(x))
        x = self.activation_func(self.fc2(x))
        x = self.activation_func(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

In [171]:
class Classifier8L(nn.Module):
    def __init__(self, input_size, layer_size, activation_func):
        super().__init__()
        
        if activation_func == 'relu':
            self.activation_func = nn.ReLU()
        elif activation_func == 'sigmoid':
            self.activation_func = nn.Sigmoid()
        
        self.fc1 = nn.Linear(input_size, layer_size)
        self.fc2 = nn.Linear(layer_size, layer_size)
        self.fc3 = nn.Linear(layer_size, layer_size)
        self.fc4 = nn.Linear(layer_size, layer_size)
        self.fc5 = nn.Linear(layer_size, layer_size)
        self.fc6 = nn.Linear(layer_size, layer_size)
        self.fc7 = nn.Linear(layer_size, layer_size)
        self.fc8 = nn.Linear(layer_size, 1)

    def forward(self, x):
        x = self.activation_func(self.fc1(x))
        x = self.activation_func(self.fc2(x))
        x = self.activation_func(self.fc3(x))
        x = self.activation_func(self.fc4(x))
        x = self.activation_func(self.fc5(x))
        x = self.activation_func(self.fc6(x))
        x = self.activation_func(self.fc7(x))
        x = torch.sigmoid(self.fc8(x))
        return x

**Построим первую модель**

In [172]:
input_size = x_train.shape[1]
first_model = Classifier2L(input_size, 32, 'relu')
criterion = nn.BCELoss()
optimizer = optim.Adam(first_model.parameters(), lr=0.001)

In [173]:
epochs = 500
for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
        outputs = first_model(inputs)
        
        loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        
        optimizer.step()
    
    if epoch % 50 == 0:
        train_predictions = (outputs >= 0.5).float()
        accuracy = (train_predictions == targets.view(-1, 1)).float().mean()
        print(f'Train accuracy at {epoch}: {accuracy.item():.4f}')

Train accuracy at 0: 0.4444


Train accuracy at 50: 0.8056
Train accuracy at 100: 0.7500
Train accuracy at 150: 0.7222
Train accuracy at 200: 0.8056
Train accuracy at 250: 0.6389
Train accuracy at 300: 0.8611
Train accuracy at 350: 0.7222
Train accuracy at 400: 0.7222
Train accuracy at 450: 0.8889


In [174]:
first_model.eval()
with torch.no_grad():
    test_outputs = first_model(x_test)
    test_predictions = (test_outputs >= 0.5).float()
    accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
print(f'Test accuracy: {accuracy.item():.4f}')

Test accuracy: 0.7973


**Теперь найдем наиболее оптимальные гиперпараметры**

In [175]:
epochs_nums = [10, 50, 150, 300]
layer_sizes = [10, 32, 64]
activation_functions = ['relu', 'sigmoid']

In [176]:
max_accuracy = 0
max_accuracy_params = {'layers_num': 0,
                       'epochs': 0,
                       'layer_size': 0,
                       'activation_function': 0}

for epochs_num in epochs_nums:
    for layer_size in layer_sizes:
        for activation_function in activation_functions:
            init_max_accuracy = max_accuracy
            # 2 слоя
            model2l = Classifier2L(input_size, layer_size, activation_function)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model2l.parameters(), lr=0.001)
            
            for epoch in range(epochs_num):
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    
                    outputs = model2l(inputs)
                    
                    loss = criterion(outputs, targets.view(-1, 1))
                    loss.backward()
                    
                    optimizer.step()
            
            model2l.eval()
            with torch.no_grad():
                test_outputs = model2l(x_test)
                test_predictions = (test_outputs >= 0.5).float()
                accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
            accuracy2l = accuracy
            
            if accuracy2l > max_accuracy:
                max_accuracy = accuracy2l
                max_accuracy_params['layers_num'] = 2
            
            # 4 слоя
            model4l = Classifier4L(input_size, layer_size, activation_function)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model4l.parameters(), lr=0.001)
            
            for epoch in range(epochs_num):
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    
                    outputs = model4l(inputs)
                    
                    loss = criterion(outputs, targets.view(-1, 1))
                    loss.backward()
                    
                    optimizer.step()
            
            model4l.eval()
            with torch.no_grad():
                test_outputs = model4l(x_test)
                test_predictions = (test_outputs >= 0.5).float()
                accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
            accuracy4l = accuracy
            
            if accuracy4l > max_accuracy:
                max_accuracy = accuracy4l
                max_accuracy_params['layers_num'] = 4
            
            # 8 слоев
            model8l = Classifier8L(input_size, layer_size, activation_function)
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model8l.parameters(), lr=0.001)
            
            for epoch in range(epochs_num):
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    
                    outputs = model8l(inputs)
                    
                    loss = criterion(outputs, targets.view(-1, 1))
                    loss.backward()
                    
                    optimizer.step()
            
            model8l.eval()
            with torch.no_grad():
                test_outputs = model8l(x_test)
                test_predictions = (test_outputs >= 0.5).float()
                accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
            accuracy8l = accuracy
            
            if accuracy8l > max_accuracy:
                max_accuracy = accuracy8l
                max_accuracy_params['layers_num'] = 8
            
            if max_accuracy != init_max_accuracy:
                max_accuracy_params['epochs'] = epochs_num
                max_accuracy_params['layer_size'] = layer_size
                max_accuracy_params['activation_function'] = activation_function

In [177]:
print(max_accuracy)
print(max_accuracy_params)

tensor(0.7973)
{'layers_num': 4, 'epochs': 300, 'layer_size': 32, 'activation_function': 'sigmoid'}


In [178]:
mse_max_accuracy = 0
mse_max_accuracy_params = {'layers_num': 0,
                           'epochs': 0,
                           'layer_size': 0,
                           'activation_function': 0}

for epochs_num in epochs_nums:
    for layer_size in layer_sizes:
        for activation_function in activation_functions:
            init_max_accuracy = mse_max_accuracy
            # 2 слоя
            model2l = Classifier2L(input_size, layer_size, activation_function)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model2l.parameters(), lr=0.001)
            
            for epoch in range(epochs_num):
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    
                    outputs = model2l(inputs)
                    
                    loss = criterion(outputs, targets.view(-1, 1))
                    loss.backward()
                    
                    optimizer.step()
            
            model2l.eval()
            with torch.no_grad():
                test_outputs = model2l(x_test)
                test_predictions = (test_outputs >= 0.5).float()
                accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
            accuracy2l = accuracy
            
            if accuracy2l > mse_max_accuracy:
                mse_max_accuracy = accuracy2l
                mse_max_accuracy_params['layers_num'] = 2
            
            # 4 слоя
            model4l = Classifier4L(input_size, layer_size, activation_function)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model4l.parameters(), lr=0.001)
            
            for epoch in range(epochs_num):
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    
                    outputs = model4l(inputs)
                    
                    loss = criterion(outputs, targets.view(-1, 1))
                    loss.backward()
                    
                    optimizer.step()
            
            model4l.eval()
            with torch.no_grad():
                test_outputs = model4l(x_test)
                test_predictions = (test_outputs >= 0.5).float()
                accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
            accuracy4l = accuracy
            
            if accuracy4l > mse_max_accuracy:
                mse_max_accuracy = accuracy4l
                mse_max_accuracy_params['layers_num'] = 4
            
            # 8 слоев
            model8l = Classifier8L(input_size, layer_size, activation_function)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model8l.parameters(), lr=0.001)
            
            for epoch in range(epochs_num):
                for inputs, targets in train_loader:
                    optimizer.zero_grad()
                    
                    outputs = model8l(inputs)
                    
                    loss = criterion(outputs, targets.view(-1, 1))
                    loss.backward()
                    
                    optimizer.step()
            
            model8l.eval()
            with torch.no_grad():
                test_outputs = model8l(x_test)
                test_predictions = (test_outputs >= 0.5).float()
                accuracy = (test_predictions == y_test.view(-1, 1)).float().mean()
            accuracy8l = accuracy
            
            if accuracy8l > mse_max_accuracy:
                mse_max_accuracy = accuracy8l
                mse_max_accuracy_params['layers_num'] = 8
            
            if mse_max_accuracy != init_max_accuracy:
                mse_max_accuracy_params['epochs'] = epochs_num
                mse_max_accuracy_params['layer_size'] = layer_size
                mse_max_accuracy_params['activation_function'] = activation_function

In [181]:
print(mse_max_accuracy)
print(mse_max_accuracy_params)

tensor(0.8018)
{'layers_num': 2, 'epochs': 300, 'layer_size': 64, 'activation_function': 'sigmoid'}


In [184]:
xgb_model = xgb.XGBClassifier(n_estimators=100)
xgb_model.fit(x_train, y_train)
xgb_prediction = xgb_model.predict(x_test)
xgb_accuracy = accuracy_score(y_test, xgb_prediction)

print(xgb_accuracy)

0.7866868381240545
